### Bibliotheken einlesen

Für das Aufbereiten und Visualisieren, sowie für die Erstellung der Machine Learning Modelle werden verschiedene Bibliotheken benötigt. Folgende Python Bibiliotheken werden zur Durchführung der Analysen verwendet:

##### *pandas*

Mit der [**pandas**](https://pandas.pydata.org/) Bibiliothek können Daten aus verschiedenen Formaten in ein Data Frame eingelesen werden. Es stehen weiterhin Funktionen für die Datenbereinigung, für das Aggregieren oder Transformieren von Daten und anderen Dingen zur Verfügung.

---

##### matplotlib

[**matplotlib**](https://matplotlib.org/) ist eine weit verbreitete Python Bibliothek mit der man verschiedene Charts erstellen kann. Sie bietet viele Konfigurationsmöglichkeiten, um auch komplexe Darstellungen zu ermöglichen

---

##### *seaborn*

Die [**seaborn**](https://seaborn.pydata.org/) Bibliothek baut auf der Library matplotlib auf und ermöglicht es, mit einfacher Syntax anschauliche Datenvisualisierungen zu erzeugen. Im Vergleich zu matplotlib wirken die Grafiken moderner und sind mit weniger Kommandos zu erstellen.

---

##### *plotly*

Die [**plotly**](https://plotly.com/) ..

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots


import warnings
warnings.filterwarnings("ignore")

### Überblick über die Daten erhalten

Im nächsten Schritt wird sich ein Überblick über die Daten verschafft. Dabei werden die einzelnen Variablen betrachtet und beschrieben.

Dazu werden die Daten aus der CSV-Datei *train.csv* in ein pandas DataFrame *df* eingelesen. Anschließend wird mit dem Aufruf des DataFrame-Namen eine Übersicht des Datasets ausgegeben.

In [ ]:
df = pd.read_csv('heart_2020_cleaned.csv')
df

Aus der Übersicht geht hervor, dass der Datensatz aus insgesamt 319795 Zeilen und 18 Spalten besteht. Sowohl die Anzahl Zeilen also auch die Anzahl der Spalten eignet sich sehr gut für die Entwicklung eines aussagekräftigen Modells. Für ein besseres Verständnis des Datasets werden im folgenden die Beschreibungen der Spalten aufgeführt:

| Variable      | Beschreibung |
| :-----------   | :-----------  |
| **HeartDisease**  | Befragte, die jemals eine koronare Herzkrankheit (KHK) oder einen Myokardinfarkt (MI) hatten   |
| BMI     | Body Mass Index (BMI)         |
| Smoking | Haben die Probanden in ihrem Leben mehr als 100 Zigaretten geraucht? |
| AlcoholDrinking | Starke Trinker (erwachsene Männer mit mehr als 14 Getränken pro Woche und erwachsene Frauen mit mehr als 7 Getränken pro Woche) |
| Stroke | Wurde dem Probanden jemals gesagt, dass er einen Schlaganfall hatte? |
| PhysicalHealth | An wie viele der letzten 30 Tage hatten die Probanden physische körperliche Beschwerden |
| MentalHealth | An wie viele der letzten 30 Tage hatten die Probanden psychische Beschwerden |
| DiffWalking | Haben die Probanden starke Beschwerden, Treppen zu steigen? |
| Sex | Geschlecht der Probanden |
| AgeCategory | Alterskategorie (insgesamt 14 Kategorien) |
| Race | Rasse / Ethnizität |
| Diabetic | Hatte der Proband jemals Diabetes? |
| PhysicalActivity | Hat der Proband in den letzten 30 Tagen Sport gemacht |
| GenHealth | Wie schätzt der Proband seine Gesundheite ein -> Exzellent / Sehr gut / gut / angemessen / schlecht |
| SleepTime | Durschnittliche Schlafzeit in Stunden |
| Asthma | Hatte der Proband jemals Asthma? |
| KidneyDisease | Wurde dem Probanden jemals gesagt, dass Sie eine Nierenerkrankung haben, ausgenommen Nierensteine, Blasenentzündung oder Inkontinenz? |
| SkinCancer | Hatte der Proband jemals Hautkrebs? |


Anhand der Daten der verschiedenen Spalten ist zu erkennen, dass das Dataset sowohl kategorische, als auch kontinuierliche Spalten besitzt. Für die unterschiedlichen Spaltentypen sind unterschiedliche Analysen und Visualisierungen sinnvoll. Daher wird im Folgenden eine **Aufteilung der Spalten in Arrays** vorgenommen, um sie im Anschluss getrennt analysieren zu können.

Das Array *cat_cols* hält alle kategorischen Spalten, das Array *con_cols* alle kontinuierlichen Spalten.
Zusätzlich wird ein Array angelegt mit der Zielvariable *price_range*. Diese wird später für die Vorhersage des Preises eines Smartphones genutzt.

In [ ]:
## AgeCategory zu einer kontinuierlichen Variable konvertieren

encode_AgeCategory = {'55-59':57, '80 or older':80, '65-69':67, '75-79':77,'40-44':42,'70-74':72,'60-64':62, '50-54':52,'45-49':47,'18-24':21,'35-39':37, '30-34':32,'25-29':27}
df['AgeCategory'] = df['AgeCategory'].apply(lambda x: encode_AgeCategory[x])
df['AgeCategory'] = df['AgeCategory'].astype('float')

In [ ]:
cat_cols = ['Smoking','AlcoholDrinking','Stroke','DiffWalking','Sex','Race',
            'Diabetic', 'PhysicalActivity', 'GenHealth', 'Asthma', 'KidneyDisease', 'SkinCancer']

con_cols = ["BMI","PhysicalHealth","MentalHealth","AgeCategory", "SleepTime"]

target_col = ["HeartDisease"]

Bevor die einzelnen Spalten analysiert werden, werden zunächst noch einige Analysen über den Datensatz insgesamt vorgenommen. So werden beispielsweise die eindeutigen Werte pro Spalte betrachtet, oder ob Spalten vorliegen, die null-Werte enthalten.

Ziel ist es, die Qualtät des Datensatzes zu ermitteln. Fehlende oder fälschliche Daten können im Verlauf der Analyse oder auch bei der Erstellung der Modelle zu schlechten oder falschen Ergebnissen führen.

Hierzu wird eine Übersicht als Tabelle erstellt, in der der Datentyp, die fehlenden und eindeutigen Werte, sowie das Minimum und Maximum pro Spalte dargestellt wird. Zur Erstellung dieser Werte bietet die Bibliothek *pandas* einige hilfreiche Funktionen.

In [ ]:
## Datentyp
data_types = pd.DataFrame(
    df.dtypes,
    columns=['Datentyp']
)

## Fehlende Werte
missing_data = pd.DataFrame(
    df.isnull().sum(),
    columns=['Fehlende Werte']
)

## Eindeutige Werte
unique_values = pd.DataFrame(
    columns=['Eindeutige Werte']
)
for row in list(df.columns.values):
    unique_values.loc[row] = [df[row].nunique()]

## Minimum
minimum_values = pd.DataFrame(
    columns=['Minimaler Wert']
)
for row in list(df.columns.values):
    minimum_values.loc[row] = [df[row].min()]

## Maximum
maximum_values = pd.DataFrame(
    columns=['Maximaler Wert']
)
for row in list(df.columns.values):
    maximum_values.loc[row] = [df[row].max()]


dq_report = data_types.join(missing_data).join(unique_values).join(minimum_values).join(maximum_values)
dq_report

Folgende **Erkenntnisse** gehen aus der oberen Übersicht hervor:

1. Nahezu alle Spalten besitzen den Datentyp int64. Lediglich die Spalten clock_speed und m_dep sind float64-Spalten. Das gesamte Dataset besteht demnach aus Zahlen. Es gibt also keine Text-Spalten oder ähnliches.

2. Keine der Spalten weißt fehlende Werte auf. Im Dataset liegen demnach keine Einträge mit null-Werte vor.

3. Es gibt Spalten im Dataset mit lediglich 2 eindeutigen Werte. Dies deutet auf Boolean-Spalten hin. Darüber hinaus gibt es Spalten mit nur wenigen Ausprägungen, die vermutlich kategorische Werte abbilden. Zuletzt gibt es Spalten mit sehr viele eindeutigen Werten, die auf kontinuierliche Werte hindeuten.

4. Die Minium und Maximum Werte der Spalten geben den Wertebereich an.

##### Bewertung

Insgesamt weißt das Dataset eine sehr gute Qualität auf. Es liegen keine fehlenden Werte vor, die das Ergebnis der Vorhersage oder der Analysen beeinträchtigen könnten. Darüber hinaus können im Dataset ausschließlich mit Zahlen gearbeitet werden. Es muss also keine Konvertierung von Textspalten oder ähnliches vorgenommen werden.

## Unvariate Analyse der Spalten

Nachdem eine Überblick über das Dataset gegeben wurde, werden nun Spalten des Dataset einzeln betrachtet und statistische Auswertungen erstellt. Diesen Vorgang nennt man auch "Unvariate Analyse". Es werden also noch keine Spalten in Beziehung gesetzt.
Zu Analyse werden die kategorischen und kontinuierlichen Spalten gesondert betrachtet, wie bereits eingangs erwähnt. Es bieten sich pro Spaltentyp unterschiedliche Visualisierungen und Analysen an.

Zunächst werden die **kategorischen Spalten** mithilfe von Bar-Charts betrachtet. Hierzu werden alle kategorischen Spalten in einer gemeinsamen Übersicht mithilfe von Subploty der Plotly-Bibliothek dargestellt.

In [ ]:
fig = make_subplots(rows=3, cols=4, subplot_titles=("Smoking", "AlcoholDrinking", "Stroke", "DiffWalking", "Sex", "AgeCategory",
                                                    "Race", "Diabetic", "PhysicalActivity", "GenHealth", "Asthma",
                                                    "KidneyDisease", "SkinCancer"))


fig.add_trace(go.Histogram(histfunc="count",  x=df['Smoking']),
              row=1, col=1)
fig.add_trace(go.Histogram(histfunc="count",  x=df['AlcoholDrinking']),
              row=1, col=2)
fig.add_trace(go.Histogram(histfunc="count",  x=df['Stroke']),
              row=1, col=3)
fig.add_trace(go.Histogram(histfunc="count",  x=df['DiffWalking']),
              row=1, col=4)
fig.add_trace(go.Histogram(histfunc="count",  x=df['Sex']),
              row=2, col=1)
fig.add_trace(go.Histogram(histfunc="count",  x=df['Race']),
              row=2, col=2)
fig.add_trace(go.Histogram(histfunc="count",  x=df['Diabetic']),
              row=2, col=3)
fig.add_trace(go.Histogram(histfunc="count",  x=df['PhysicalActivity']),
              row=2, col=4)
fig.add_trace(go.Histogram(histfunc="count",  x=df['GenHealth']),
              row=3, col=1)
fig.add_trace(go.Histogram(histfunc="count",  x=df['Asthma']),
              row=3, col=2)
fig.add_trace(go.Histogram(histfunc="count",  x=df['KidneyDisease']),
              row=3, col=3)
fig.add_trace(go.Histogram(histfunc="count",  x=df['SkinCancer']),
              row=3, col=4)

fig.update_layout(height=700, title_text="Kategorische Variablen")
fig.show()

Für die Betrachtung der kontinuierlichen Variablen bieten sich Boxplots an. Boxplots sind Diagramme, die zur graphischen Darstellung der Verteilung von mindestens ordinalskalierten Merkmalen verwendet werden. Es fasst dabei verschiedene Streuungs- und Lagemaße in einer Darstellung zusammen. Ein Box-Plot soll schnell einen Eindruck darüber vermitteln, in welchem Bereich die Daten liegen und wie sie sich über diesen Bereich verteilen.

Ein Boxplot fasst folgende Werte in einer Darstellung zusammen:
- Minimum
- Unteres Quartil
- Median
- Oberes Quartil
- Maximum
- Spannweite
- Interquartilsabstand (IQR)

Darüber hinaus werden extreme Ausreißer (3x IQR) in den Daten als Punkte dargestellt.

In [ ]:
## BMI
px.histogram(df, x="BMI", height=300)

px.histogram(df, x="PhysicalHealth", height=300)
px.histogram(df, x="MentalHealth", height=300)
px.histogram(df, x="AgeCategory", height=300)
px.histogram(df, x="SleepTime", height=300)

## Bivariate Analyse der Spalten

Im Anschluss an die unvariate Analyse folgt die bivariate Analyse der Spalten. Der Fokus liegt hier in der Suche nach Beziehungen zwischen 2 oder mehr Spalten. Genauer gesagt ist das Ziel, eine Beziehung zischen der Zielvariabel *HeartDisease* und einer beschreibenden Variable wie z.B. *Smoking* zu finden. Es wird also eine Art empirische Beziehung zwischen den Variablen ermittelt.

Außerdem werden Korrelationen zwischen den beschreibenden Variablen ermittelt. Eine hohe Korrelation zweier Variablen sagt aus, dass die eine Variable die andere bedingt.

Vor der bivariaten Analyse werden allerdings noch Hypothesen aufgestellt, die eine Richtung bei der Analyse der Daten vorgeben. Man stellt also Vermutungen zu den Daten auf, die im Anschluss mittels Analysen und Visualisierung belegt oder wiederlegt werden.

**Hypthesen**:

1.

In [ ]:
plt.subplot(231)
sns.countplot(x="Smoking", hue='HeartDisease', data=df)
plt.subplot(232)
sns.countplot(x="AlcoholDrinking", hue='HeartDisease', data=df)
plt.subplot(233)
sns.countplot(x="Stroke", hue='HeartDisease', data=df)

In [ ]:
plt.subplot(234)
sns.countplot(x="DiffWalking", hue='HeartDisease', data=df)
plt.subplot(235)
sns.countplot(x="Sex", hue='HeartDisease', data=df)
plt.subplot(236)
sns.countplot(x="Race", hue='HeartDisease', data=df)

In [ ]:
plt.subplot(234)
sns.countplot(x="Diabetic", hue='HeartDisease', data=df)
plt.subplot(235)
sns.countplot(x="PhysicalActivity", hue='HeartDisease', data=df)
plt.subplot(236)
sns.countplot(x="GenHealth", hue='HeartDisease', data=df)

In [ ]:
plt.subplot(234)
sns.countplot(x="Asthma", hue='HeartDisease', data=df)
plt.subplot(235)
sns.countplot(x="KidneyDisease", hue='HeartDisease', data=df)
plt.subplot(236)
sns.countplot(x="SkinCancer", hue='HeartDisease', data=df)